In [1]:
from bisect import bisect_left
import sys
sys.path.append(r"C:\Users\jarra\Desktop\Masters\Heatwave_Project")
import pandas as pd
import PT13_Functions_For_Masters_New_Test as HW_Func

import Strictly_Functions_Project as SFP
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau


import scipy
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import warnings
warnings.filterwarnings("ignore")
#RMSE 
from sklearn.metrics import mean_squared_error
from math import sqrt
from datetime import datetime
import seaborn as sns
import math

In [2]:


def Simple_QQ_Regression(Q_step, Historical, Present, Hist_Dates, Pres_Date):
    number = Q_step
    Hist_All = Historical
    Present = Present
    Hist_QQ_Dates_St = Hist_Dates[0]
    Hist_QQ_Dates_En = Hist_Dates[1]

    Pres_QQ_Date_St = Pres_Date[0]
    Pres_QQ_Date_En = Pres_Date[1]
    #^ call in using function
    #^ call in using function

    Historical_30 = Hist_All.loc[Hist_QQ_Dates_St:Hist_QQ_Dates_En].reset_index()
    Present_30 = Present.loc[Pres_QQ_Date_St:Pres_QQ_Date_En].reset_index()



    #Select a nparange value lemgth that goes from Q0 to Q1, and produce the quantiles
    QPRE = Present_30.quantile(np.arange(0,1+number,number)).round(4)
    QPRE = QPRE.rename_axis('Quantile').reset_index()

    QHIS = Historical_30.quantile(np.arange(0,1+number,number)).round(4)
    QHIS = QHIS.rename_axis('Quantile').reset_index()


    #Hist_All = Historical_All.reset_index()
    #This above is the full range of historical data

    #What we will do is append it to max and min values before combiniing with dat
    Hist_Updated_Max = []
    Hist_Updated_Min = []
    Hist_Updated_Date = []



    Hist_All = Hist_All.reset_index()
    #For loop for all dates 
    for i in range(0,len(Hist_All)):
        Hist_Updated_Date.append(Hist_All['date'].loc[i])


        #Now get all the information from the Q-Q data for max and min for each date
        #MAX

        #If data shows a nan value set the updated value to nan
        if (math.isnan(Hist_All['tmax'].loc[i])== True):
            Hist_Updated_Max.append(np.NaN)
        else:
            #Set Temp old 
            Temp_Old = Hist_All['tmax'].loc[i]



            #So now we get the closest value for the max:
            Column = ['tmax']

            #This finds the value where the Q-Hist of the tmax is the minimum it can be for the tmax value presented
            Min_val = np.abs(QHIS[Column] - Temp_Old).min()

            #This finds the quantile*10^5 or by the decimla place you use to find the tmax
            closest_index =  QHIS[np.abs(QHIS[Column]- Temp_Old) == Min_val].stack().idxmin()

            #Now this will use the index to find the Present Vlaue to updayte the historical value to using the index/quantile*10^%
            Hist_Updated_Max.append(QPRE[Column].loc[closest_index[0]].values[0])

        #Now get all the information from the Q-Q data for max and min for each date
        #MIN

        #If data shows a nan value set the updated value to nan
        if (math.isnan(Hist_All['tmin'].loc[i])== True):
            Hist_Updated_Min.append(np.NaN)
        else:
            #Set Temp old 
            Temp_Old = Hist_All['tmin'].loc[i]


            #So now we get the closest value for the max:
            Column = ['tmin']

            #This finds the value where the Q-Hist of the tmax is the minimum it can be for the tmax value presented
            Min_val = np.abs(QHIS[Column] - Temp_Old).min()

            #This finds the quantile*10^5 or by the decimla place you use to find the tmax
            closest_index =  QHIS[np.abs(QHIS[Column]- Temp_Old) == Min_val].stack().idxmin()

            #Now this will use the index to find the Present Vlaue to updayte the historical value to using the index/quantile*10^%
            Hist_Updated_Min.append(QPRE[Column].loc[closest_index[0]].values[0])
    Hist_Updated_Date = pd.DataFrame(Hist_Updated_Date, columns=['date'])
    Hist_Updated_Max = pd.DataFrame(Hist_Updated_Max, columns=['tmax'])
    Hist_Updated_Min = pd.DataFrame(Hist_Updated_Min, columns=['tmin'])


    #Now combine altogether
    Hist_Updated = pd.concat([Hist_Updated_Date, Hist_Updated_Max, Hist_Updated_Min], axis = 1)
    return(Hist_Updated)
        


In [ ]:
Hist_Updated

In [3]:
#Now putting it all together

#Now lets load thedatasets in
Hist_Max = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\FOR HOMOGENISATION\Tmax_Est_1830_1875.csv")
Hist_Min = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\FOR HOMOGENISATION\Tmin_Est_1830_1875.csv")
Present = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\FOR HOMOGENISATION\ACORN_SAT_1880_2021.csv")

HMax_Col = Hist_Max.columns
HMin_Col = Hist_Min.columns
Present = Present
length_col = len(HMax_Col)

i =1

#Fix the data so its only one trail we are looking at first, this is the columns of the max and min
Hist_Max_Col = Hist_Max[['date',HMax_Col[i]]]
Hist_Min_Col = Hist_Min[['date',HMin_Col[i]]]

#We rename them to they have tmax and tmin
Hist_Max_Col = Hist_Max_Col.rename(columns={HMax_Col[i]:'tmax'})
Hist_Min_Col = Hist_Min_Col.rename(columns={HMin_Col[i]:'tmin'})

#We add the historical datasets together
Hist_Together = pd.concat([Hist_Max_Col['date'], Hist_Max_Col['tmax'],Hist_Min_Col['tmin']],axis = 1)

Hist_Together =Hist_Together.round(4).set_index('date')
Present = Present.set_index('date')

#This is the first trial
Hist_Corrected = Simple_QQ_Regression(0.0001, Hist_Together, Present, ['1846-01-01',"1875-12-31"], ['1880-01-01',"1909-12-31"])

#This is the corrected temps
Hist_Corrected = Hist_Corrected.rename(columns={'tmax':HMax_Col[i]})
Hist_Corrected = Hist_Corrected.rename(columns={'tmin':HMin_Col[i]})

#This is the full list
Hist_Corrected_Dict_Full = Hist_Corrected



for i in range(2,length_col):#length_col):
    print(i)
    #Fix the data so its only one trail we are looking at first
    Hist_Max_Col = Hist_Max[['date',HMax_Col[i]]]
    Hist_Min_Col = Hist_Min[['date',HMin_Col[i]]]


    Hist_Max_Col = Hist_Max_Col.rename(columns={HMax_Col[i]:'tmax'})
    Hist_Min_Col = Hist_Min_Col.rename(columns={HMin_Col[i]:'tmin'})

    Hist_Together = pd.concat([Hist_Max_Col['date'], Hist_Max_Col['tmax'],Hist_Min_Col['tmin']],axis = 1)
    Hist_Together =Hist_Together.round(4).set_index('date')

    Hist_Corrected = Simple_QQ_Regression(0.0001, Hist_Together, Present, ['1846-01-01',"1875-12-31"], ['1880-01-01',"1909-12-31"])
    
    Hist_Corrected = Hist_Corrected.rename(columns={'tmax':HMax_Col[i]})
    Hist_Corrected = Hist_Corrected.rename(columns={'tmin':HMin_Col[i]})
    
    Hist_Corrected_Dict_Full = pd.merge(Hist_Corrected_Dict_Full,Hist_Corrected,on = 'date')



Hist_Corrected_Dict_Full.to_csv(r"C:\Users\jarra\Desktop\Perth_1830_1875_Corrected.csv")



2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

In [4]:
#Now lets load thedatasets in
Hist_Max = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\FOR HOMOGENISATION\Tmax_Est_1830_1875.csv")
Hist_Min = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\FOR HOMOGENISATION\Tmin_Est_1830_1875.csv")
Present = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\FOR HOMOGENISATION\ACORN_SAT_1880_2021.csv")

HMax_Col = Hist_Max.columns
HMin_Col = Hist_Min.columns
Present = Present
length_col = len(HMin_Col)


In [5]:
length_col

1008

In [4]:
Hist_Corrected_Dict_Full

date  Max Mean  Min Mean  Max Median  Min Median  \
0      1830-04-16 00:00:00    25.100       NaN     25.1000         NaN   
1      1830-04-17 00:00:00    25.800      10.8     25.8000        10.6   
2      1830-04-18 00:00:00    28.200      13.3     28.1000        13.0   
3      1830-04-19 00:00:00    26.500      14.0     26.5000        14.0   
4      1830-04-20 00:00:00    24.400       7.8     24.4000         7.8   
...                    ...       ...       ...         ...         ...   
16692  1875-12-28 00:00:00    38.200       NaN     38.1715         NaN   
16693  1875-12-29 00:00:00    35.407      18.3     35.4070        18.2   
16694  1875-12-30 00:00:00    33.000      14.6     33.0000        13.4   
16695  1875-12-31 00:00:00    25.700      14.4     25.7000        14.2   
16696  1876-01-01 00:00:00       NaN      14.4         NaN        14.5   

       Max Lower CI (60%)  Min Lower CI (60%)  Max Upper CI (60%)  \
0                    25.1                 NaN                25.1   
1                    25.8                10.9                25.8   
2                    28.1                13.4                28.3   
3                    26.5                14.1                26.5   
4                    24.4                 7.9                24.3   
...                   ...                 ...                 ...   
16692                38.2                 NaN                38.1   
16693                35.5                18.3                35.6   
16694                33.0                13.5                33.0   
16695                25.7                14.4                25.6   
16696                 NaN                14.9                 NaN   

       Min Upper CI (60%)  Max Lower CI (90%)  ...  Max_996  Min_996  Max_997  \
0                     NaN              25.100  ...  25.1000      NaN  25.1000   
1                    10.4              25.800  ...  25.8000     10.6  25.8000   
2                    12.8              27.800  ...  28.4000     13.0  28.3000   
3                    13.9              26.400  ...  26.2000     14.0  26.3000   
4                     7.7              24.400  ...  24.4000      7.8  24.3605   
...                   ...                 ...  ...      ...      ...      ...   
16692                 NaN              38.200  ...  38.1715      NaN  38.2000   
16693                18.1              35.502  ...  35.7000     18.1  35.8000   
16694                16.8              33.000  ...  33.0000     13.4  33.0000   
16695                14.5              25.800  ...  25.7000     14.1  25.8000   
16696                14.4                 NaN  ...      NaN     14.4      NaN   

       Min_997  Max_998  Min_998  Max_999  Min_999  Max_1000  Min_1000  
0          NaN     25.1      NaN     25.1      NaN     25.10       NaN  
1         10.6     25.8     10.7     25.8     10.9     25.80      10.6  
2         13.0     27.8     13.0     28.2     13.4     28.10      13.2  
3         14.0     26.6     13.9     26.4     14.1     26.20      13.9  
4          7.7     24.4      7.8     24.4      7.8     24.40       7.8  
...        ...      ...      ...      ...      ...       ...       ...  
16692      NaN     37.8      NaN     38.0      NaN     38.10       NaN  
16693     18.1     35.4     18.7     35.4     18.1     35.64      18.7  
16694     13.4     33.0     17.3     32.9     13.4     33.00      17.2  
16695     14.1     25.7     15.3     25.7     14.1     25.70      15.3  
16696     14.4      NaN     14.4      NaN     14.5       NaN      14.4  

[16697 rows x 2015 columns]